In [ ]:
using Plots
using FFTW

L = 100
N = 1000
dx = L/N
dt = 1e-5

x = (L/N) .* collect(0:N-1)
k = 2*π / L .* collect(0:N÷2)
k² = k.*k

F = plan_rfft(x)
B = plan_irfft(k.*im, N)

function antialiasing!( F )
    F[end-(N÷6)+1:end] .= 0.
end

function euler!(Fdφ, Fφ, φ)
    Fdφ .= -k² .* (k².*Fφ .- F * (φ .- φ.^3 ))
    Fφ .+= Fdφ.*dt
    antialiasing!(Fφ)
    φ .= B*Fφ
end

In [ ]:
φ = cos.(2  * (2π/L) .* x)  * 0.1

dφ = zeros(N)
Fφ = F*φ 
Fdφ = F*dφ 
c = "lightblue"
p = plot(x, φ, color=c)
display(p)

In [ ]:
M = 1_000_000
p = plot(x, φ, color=c)
for t in 1:M
    euler!(Fdφ, Fφ, φ)
    if t%(M÷10) == 0
        print(100*t/M,"% \n")
        plot!(x, φ, color=c)
        @assert !any(isnan.(Fφ))
    end
end
display(p)

In [ ]:
not